# Разбор возможностей Hugging Face с использованием бесплатных моделей
Hugging Face предлагает множество бесплатных предобученных моделей, которые можно использовать для обработки естественного языка (NLP). В данном руководстве посмотрим, как загружать и применять эти модели, используя библиотеку transformers.

Hugging Face предлагает библиотеку transformers для работы с моделями и datasets для работы с открытыми наборами данных.

In [ ]:
!pip install transformers datasets torch

### 2. Загрузка бесплатной модели для работы с текстом
Для начала загрузим предобученную модель BERT и её токенизатор:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Выбираем предобученную бесплатную модель
model_name = "distilbert-base-uncased"

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("Модель и токенизатор успешно загружены!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель и токенизатор успешно загружены!


Здесь мы используем DistilBERT, который является облегчённой версией BERT, обученной без потери качества.

 ## 3. Токенизация текста
Токенизируем входной текст перед подачей в модель:

In [ ]:
text = "Hugging Face makes NLP easy!"
inputs = tokenizer(text, return_tensors="pt")
print(inputs)


{'input_ids': tensor([[  101, 17662,  2227,  3084, 17953,  2361,  3733,   999,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


input_ids — числовые представления слов, attention_mask указывает, какие токены учитывать при обработке.

## 4. Работа с моделью: Анализ тональности
Теперь передадим данные в модель:



In [ ]:
from torch.nn.functional import softmax

outputs = model(**inputs)
probs = softmax(outputs.logits, dim=1)

print(f"Положительная тональность: {probs[0][1].item():.4f}")
print(f"Отрицательная тональность: {probs[0][0].item():.4f}")


Положительная тональность: 0.5175
Отрицательная тональность: 0.4825


Результат: модель выдаст вероятности тональности (положительная или отрицательная).

## 5. Использование конвейеров (pipelines)
В transformers есть удобный инструмент pipeline, который автоматически подбирает нужную модель и обрабатывает текст без сложной настройки. Поддерживаются разные задачи, такие как анализ тональности, генерация текста, ответы на вопросы и др. Попробуем использовать конвейеры для анализа тональности, генерации текста и ответов на вопросы.
Название конвейера указывается в качестве аргумента pipeline. Например, для анализа тональности используется "sentiment-analysis". Для генерации текста — "text-generation". Для ответов на вопросы — "question-answering". Для сжатия текста — "summarization". Для перевода — "translation_xx_to_yy" (где xx и yy — языки перевода).

✅ Пример анализа тональности:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
print(classifier("Hugging Face is amazing!"))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998840093612671}]


## Пример генерации текста с GPT-2:

In [ ]:
text_generator = pipeline("text-generation", model="gpt2")
print(text_generator("Once upon a time", max_length=50))


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Once upon a time in my life, I was a very young woman, and I was so young that I didn't really know what to do with myself after I ended up in a place that I could be myself and have that ability to be myself.\n\nI think it's important to remember that I was raised by my mother and my father, and that there is no way that I and my siblings are going to be able to meet the same fate as we do. There is a very big difference between being a man and a woman. I'm not going to get into why. I'm just going to point this out.\n\nI think the most important thing is to remember that we are human beings, and we can be. We are not what we are. You don't have to be a man to be a woman. You don't have to be very strong or strong. You just have to be able to live and love, and to be a good person like you are and be an interesting person you are. And it's the same thing in the way that the person who is in the room, the person who is the speaker, the person who is the voice, the 

## Пример ответа на вопросы:

In [ ]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

context = "Hugging Face is a company that specializes in Natural Language Processing."
question = "What does Hugging Face specialize in?"

result = qa_pipeline(question=question, context=context)
print(result)


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


{'score': 0.9702426256699255, 'start': 46, 'end': 73, 'answer': 'Natural Language Processing'}


## 6. Как найти и использовать другие бесплатные модели?
Hugging Face Hub (https://huggingface.co/models) содержит тысячи бесплатных моделей. Можно искать по ключевым словам и загружать нужную модель так:


    model_name = "facebook/bart-large-mnli"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

### Пример нулевого (zero-shot) классификатора:

Такой классификатор, может предсказать, что образец соответствует некоторому классу, даже если такие образцы не наблюдались во время обучения

    zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    
    sequence = "I want to book a flight to Paris."
    labels = ["travel", "finance", "sports"]
    
    print(zero_shot_classifier(sequence, candidate_labels=labels))




In [ ]:
model_name = "facebook/bart-large-mnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:


zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sequence = "I want to book a flight to Paris."
labels = ["travel", "finance", "sports"]

print(zero_shot_classifier(sequence, candidate_labels=labels))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


{'sequence': 'I want to book a flight to Paris.', 'labels': ['travel', 'finance', 'sports'], 'scores': [0.9830968976020813, 0.011268587782979012, 0.005634504370391369]}


### ЗАДАНИЕ
1. Выберите другую модель из Hugging Face Hub и загрузите её. Модель должна поддерживать русский язык
2. Выполните анализ тональности текста с использованием новой модели.
В качестве примеров используйте отзывы на фильмы или продукты на русском языке.
3. Выберите модель для ответов на вопросы и выполните задание 6 для русского языка.
4. Найдите модель для русского языка и выполните суммаризацию (сжатие, краткий пересказ) текста первых трех глав Гарри Поттера.

    summarizer = pipeline("summarization", model="...")
5. Найдите датасет с песнями на русском языке. выполните парафраз текстов песен. _pipeline("text2text-generation" ....)_



# Что такое datasets в Hugging Face?
Hugging Face предоставляет библиотеку datasets, которая содержит тысячи бесплатных наборов данных для NLP и других задач машинного обучения.

✅ Основные преимущества datasets:
✔ Поддержка больших наборов данных (работает с памятью эффективно)
✔ Можно загружать, обрабатывать и тренировать модели
✔ Поддержка многих форматов данных (CSV, JSON, Parquet и т. д.)
✔ Готовые стандартные датасеты для NLP (SQuAD, IMDB, WikiText и др.)

# Пример загрузки набора данных
Загрузим популярный IMDb (анализ тональности):

### Что мы увидим?

Датасет состоит из тренировочного (train) и тестового (test) наборов
В каждом 25 000 примеров
Колонки text и label
text – отзыв
label – метка (0 – негативный, 1 – позитивный)

In [ ]:
from datasets import load_dataset

# Загружаем датасет IMDb
dataset = load_dataset("imdb")

# Смотрим структуру данных
print(dataset)


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


# Доступ к данным
Посмотрим первые примеры из датасета:

In [ ]:
print(dataset["train"][0])  # Вывод первого примера


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# Что такое features в Hugging Face?
В Hugging Face features – это признаки (колонки) набора данных.

💡 Как посмотреть features (структуру данных)?

In [ ]:
print(dataset["train"].features)


# Что означают features
# text: строка (сам отзыв)
# label: классификационная метка (0 – "negative", 1 – "positive")


{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}


*Features* —
это просто характеристики или поля в датасете, которые описывают, из чего состоит каждая запись. Датасет для отзывов, наверное, из какой-то библиотеки вроде Hugging Face Datasets.

*text:*
Это строка с самим текстом отзыва. То есть, это то, что человек написал — положительный или отрицательный отзыв о фильме, продукте или чём-то ещё. Например, "Фильм супер, всем советую!" или "Полный отстой, зря потратил время".

*label:*
Это метка для классификации, которая показывает sentiment отзыва. 0 значит "negative" (отрицательный, типа жалобы или критики), а 1 — "positive" (положительный, хвалебный). Это как ярлык, чтобы модель могла учиться на примерах: видит текст и понимает, к какому классу он относится.

